In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_ml_control
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
import warnings

warnings.simplefilter("ignore", UserWarning)

In [ ]:
%autoreload

import casadi
import do_mpc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from do_mpc.controller import MPC
from do_mpc.model import Model
from numpy.typing import NDArray

from training_ml_control.control import (
    build_mpc_controller,
)

from training_ml_control.environments import (
    create_cart_environment,
    create_inverted_pendulum_environment,
    simulate_environment,
)
from training_ml_control.plots import (
    animate_cart_simulation,
    animate_inverted_pendulum_simulation,
    animate_full_inverted_pendulum_simulation,
)
from training_ml_control.control import (
    build_lqr_controller,
)
from training_ml_control.model import (
    build_cart_model,
    build_inverted_pendulum_linear_model,
)

from training_ml_control.nb_utils import (
    display_array,
    show_video,
)

sns.set_theme()
plt.rcParams["figure.figsize"] = [9, 5]

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::

# Model Predictive Control

Unfortunately, the analytically convenient linear quadratic problem formulations are often not satisfactory. There are two main reasons for this:

- The system may be nonlinear, and it may be inappropriate to use for
  control purposes. Moreover, some of the control variables may be naturally
  discrete, and this is incompatible with the linear system viewpoint.
  
- There may be control and/or state constraints, which are not handled
  adequately through quadratic penalty terms in the cost function. For
  example, the motion of a car may be constrained by the presence of
  obstacles and hardware limitations.
  The solution obtained from a linear quadratic model may not be suitable for such
  a problem, because quadratic penalties treat constraints "softly"
  and may produce trajectories that violate the constraints.

Model Predictive Control (MPC) is a control algorithm based on solving an **on-line** optimal control problem. A receding horizon approach is used which can be summarized in the following steps:

1. At time $t$ and for the current state $x_t$, solve, on-line, an open-loop optimal control problem over some future interval taking account the current and future constraints.
2. Apply the first step in the optimal control sequence.
3. Repeat the procedure at time $t + 1$ using the current state $x_{t + 1}$.

:::{figure} _static/images/40_model_predictive_control_horizon.svg
Illustration of the problem solved by MPC at state $x_k$.
We minimize the cost function over the next $l$ stages.
We then apply the control of the optimizing sequence up to the control horizon.
In most cases, the control horizon is set to 1.
:::

:::{figure} _static/images/40_mpc_block_diagram.svg
MPC Block Diagram.
:::

A major difference between MPC and finite-state stochastic control problems
that are popular in the RL/artificial intelligence literature is that in MPC
the state and control spaces are continuous/infinite, such as for example
in self-driving cars, the control of aircraft and drones, or the operation of
chemical processes

## Iterative Linear Quadratic Regulator

Iterative Linear Quadratic Regulator (iLQR) is an extension of LQR control to non-linear system with non-linear quadratic costs.

The idea is to approximate the cost and dynamics as quadratic and affine, respectively, then exactly solve the resulting LQR problem.

The basic flow of the algorithm is:

1. Initialize with initial state $\mathbf{x}_0$ and initial control sequence $\mathbf{U} = \{\mathbf{u}_{0}, \mathbf{u}_{1}, \dots, \mathbf{u}_{N-1}\}$.
2. Do a forward pass using the non-linear dynamics, i.e. simulate the system using $(\mathbf{x}_0, \mathbf{U})$ to get the trajectory through state space, $\mathbf{X}$, that results from applying the control sequence $\mathbf{X}$ starting in $\mathbf{x}_0$.
3. Do a backward pass, estimate the value function and dynamics for each $(\mathbf{x}, \mathbf{u})$ in the state-space and control signal trajectories.
4. Calculate an updated control signal $\hat{\mathbf{U}}$ and evaluate cost of trajectory resulting from $(\mathbf{x}_0, \hat{\mathbf{U}})$.
   
   1. If $|(\textrm{cost}(x_0, \hat{\textbf{U}}) - \textrm{cost}(x_0, \textbf{U})| < \textrm{threshold},$ then we've converged and exit.
   2. If $\textrm{cost}(x_0, \hat{\textbf{U}}) < \textrm{cost}(x_0, \textbf{U}),$ then set $\textbf{U} = \hat{\textbf{U}},$ and change the update size to be more aggressive. Go back to step 2.
   3. If $\textrm{cost}(x_0, \hat{\textbf{U}}) \geq \textrm{cost}(x_0, \textbf{U}),$ change the update size to be more modest. Go back to step 3.

:::{note} Mathematical Derivation
:class: dropdown

Given a discrete-time non-linear system with state-space representation: 

$$
\mathbf{x}_{t+1} = f(\mathbf{x}_t, \mathbf{u}_t)
$$

we approximate it with a first-order Taylor-series expansion about nominal trajectories
$\mathbf{X} = \{\mathbf{x}_0, \dots, \mathbf{x}_N\}, \mathbf{U} = \{ \mathbf{u}_0, \dots, \mathbf{u}_N \}$:

$$
\begin{array}{lll}
\mathbf{x}_{t+1} + \delta\mathbf{x}_{t+1} &= f(\mathbf{x}_t + \delta \mathbf{x}_t, \mathbf{u}_t + \delta \mathbf{u}_t) \\ 
& \approx f(\mathbf{x}_t, \mathbf{u}_t)
+ \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t
+ \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t\\
\delta\mathbf{x}_{t+1} &= A_t\delta\mathbf{x}_t + B_t\delta\mathbf{u}_t
\end{array}
$$

with
$A_t = A(\mathbf{x}_t, \mathbf{u}_t) = \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}$ and
$B_t = B(\mathbf{x}_t, \mathbf{u}_t) = \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}$

Given a general cost function that is not linear-quadratic, we use a second-order Taylor Series Expansion to linearize the dynamics into a form common for optimal control problems:

$$
\begin{array}{lll}
J_N(\mathbf{x}_0, \mathbf{U}) &=
l_f(\mathbf{x}_N) + \sum \limits_{t=1}^{N-1} l(\mathbf{x}_t, \mathbf{u}_t)\\
&\approx \frac{1}{2} \mathbf{x}^T_N Q \mathbf{x}_N + q_N^T \mathbf{x}_N
+ \sum \limits_{t=1}^{N-1}
\frac{1}{2} \mathbf{x}^T_t Q_t \mathbf{x}_t
+ \frac{1}{2} \mathbf{u}^T_t R_t \mathbf{u}_t
+ \frac{1}{2} \mathbf{x}^T_t H_t \mathbf{u}_t
+ \frac{1}{2} \mathbf{u}^T_t H_t^T \mathbf{x}_t
+ q_t^T \mathbf{x}_t +  + r_t^T \mathbf{u}_t
\end{array}
$$

We define the following variables for convenience:

$$
\begin{array}{lll}
l_x &:= \frac{\partial l}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t} = Q_t \mathbf{x}_t + q_t
\\
l_u &:= \frac{\partial l}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t} = R_t \mathbf{u}_t + r_t
\\
l_{xx} &:= \frac{\partial^2 l}{\partial \mathbf{x}^2}|_{\mathbf{x}_t, \mathbf{u}_t} = Q_t
\\
l_{uu} &:= \frac{\partial^2 l}{\partial \mathbf{u}^2}|_{\mathbf{x}_t, \mathbf{u}_t} = R_t
\\
l_{xu} &:= \frac{\partial^2 l}{\partial \mathbf{x}\mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t} = H_t
\\
l_{ux} &:= \frac{\partial^2 l}{\partial \mathbf{u}\mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t} = H_t^T
\end{array}
$$

We can apply Bellman's Principle of Optimality to define the optimal cost-to-go:

$$
\begin{array}{lll}
V_N(\mathbf{x}_N) &= g_f(\mathbf{x}_N)\\
V_{t}(\mathbf{x}_t) &= \displaystyle \min_u {g(\mathbf{x}_{t}, \mathbf{u}_{t}) + V_{t+1}(f(\mathbf{x}_{t}, \mathbf{u}_{N-t}))}\\
V_{t}(\mathbf{x}_t) &= \displaystyle \min_u Q_{t}(\mathbf{x}_{t}, \mathbf{u}_{t})
\end{array}\\
$$

The $Q$-function is the discrete-time analogue of the Hamiltonian, sometimes known as the pseudo-Hamiltonian.

We approximate the cost-to-go function as locally quadratic near the nominal trajectory gives us (we drop the time index in some of the equations for the sake of readability):

$$
\delta V(\mathbf{x}) = s^T \delta\mathbf{x} + \frac{1}{2} \delta\mathbf{x}^T S \delta\mathbf{x}
$$

with 
$s = \frac{\partial V}{\partial \mathbf{x}}|_{\mathbf{x}},
S = \frac{\partial^2 V}{\partial \mathbf{x}^2}|_{\mathbf{x}}$

Similarily:

$$
\delta Q(\mathbf{x}, \mathbf{u}) = 
\frac{1}{2}
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}^T
\begin{bmatrix} Q_{xx} & Q_{xu} \\ Q_{ux} & Q_{uu} \end{bmatrix}
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}
+
\begin{bmatrix} Q_{x} \\ Q_{u} \end{bmatrix}^T
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}
$$

with:

$$
\begin{array}{lll}
Q_x &= l_x + s_{t+1} A_t \\
Q_u &= l_u + s_{t+1} B_t \\
Q_{xx} &= l_{xx} + A_t^T S_{t+1} A_t \\
Q_{uu} &= l_{uu} + B_t^T S_{t+1} B_t \\
Q_{ux} &= l_{ux} + B_t^T S_{t+1} A_t
\end{array}
$$

The optimal control modification $\delta\mathbf{u}^∗$ for some state perturbation $\delta\mathbf{x}$, is obtained by minimizing the quadratic model:

$$
\delta\mathbf{u}^∗(\delta\mathbf{x}) = \displaystyle\arg\min_{\delta\mathbf{u}} Q(\delta\mathbf{x}, \delta\mathbf{u}) = k + K\delta\mathbf{x}
$$

This is a locally-linear feedback policy with:

$$
k := -Q_{uu}^{-1}Q_u\\
K := -Q_{uu}^{-1}Q_{ux}
$$

Plugging this back into the expansion of $Q$, a quadratic model of $V$ is obtained. After simplification it is:

$$
\begin{array}{lll}
\Delta V &= \frac{1}{2} k^T Q_{uu} k + k^T Q_u\\
s &= Q_x + K^T Q_{uu} k + K^T Q_u + Q_{ux}^T k\\
S &= Q_{xx} + K^T Q_{uu} K + K^T Q_{ux} + Q_{ux}^T K
\end{array}
$$

Once these terms are computed, a forward pass computes a new trajectory:

$$
\begin{array}{lll}
\hat{x}_0 &= x_0\\
\hat{u}_t &= u_t + k_t + K_t(\hat{x}_t - x_t)\\
\hat{x}_{t+1} &= f(\hat{x}_t, \hat{u}_t)
\end{array}
$$

:::

[do-mpc](https://www.do-mpc.com/en/latest/) uses [CasADi](https://web.casadi.org/python-api/) (an open-source tool for nonlinear optimization and algorithmic differentiation) for the modeling part and for the different cost functions. Here's a table of useful operators:

| Operator | Description |Equation |
| --- | --- | --- |
| [casadi.sumsqr(x)](https://web.casadi.org/python-api/#casadi.casadi.sumsqr) | Squared-sum | $\sum x_i^2$ |
| [casadi.norm_2(x)](https://web.casadi.org/python-api/#casadi.casadi.norm_2) | $L_2$-norm | $\sqrt{\sum x_i^2}$ |
| [casadi.norm_1(x)](https://web.casadi.org/python-api/#casadi.casadi.norm_1) | $L_1$-norm | $\sum |x_i|$ |
| [casadi.bilin(A, x)](https://web.casadi.org/python-api/#casadi.casadi.bilin) | Quadratic Form | $x^T A x$ |
| [casadi.bilin(A, x, y)](https://web.casadi.org/python-api/#casadi.casadi.bilin) | Bilinear Form | $x^T A y$ |

In [ ]:
?? build_mpc_controller

#### Controller

First, we create an instance of the MPC class is generated with the Mass-Spring-Damper prediction model defined above.

In [ ]:
mpc = do_mpc.controller.MPC(mass_spring_damper)

We choose the finite prediction horizon `n_horizon = 20`, the time step `t_step = 0.04s` to be the same as the environment's time step. We also set the parameters of the applied discretization scheme orthogonal collocation.

In [ ]:
env = create_mass_spring_damper_environment()
mpc_params = {
    "n_horizon": 20,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

The control objective is to move the mass to a desired position (`0.1`) and keep it there.

In [ ]:
xss = np.array([0.1, 0.0])
distance_cost = 100 * casadi.norm_2(mass_spring_damper.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

We also restrict the input force.

In [ ]:
force_penalty = 1e-2
mpc.set_rterm(force=force_penalty)

#### Constraints

We apply constraints on the force. In this case, there is only an upper and lower bounds for the force.

In [ ]:
# lower and upper bounds of the input
u_max = 20
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

We can now setup the controller.

In [ ]:
mpc.setup()

#### Simulation

We set the initial state and simulate the closed-loop for 100 steps.

In [ ]:
%%capture
mpc.reset_history()
mass_spring_damper_simulator.reset_history()
x0 = np.zeros((2, 1))
mass_spring_damper_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = mass_spring_damper_simulator.make_step(u0)

In [ ]:
animate_mass_spring_damper_simulation(mpc.data)

#### Evaluation

Finally, we evaluate the controller on the actual environment.

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(2)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 100
env = create_mass_spring_damper_environment(
    render_mode=render_mode, max_steps=max_steps
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_mass_spring_damper_simulation(mpc.data)

:::{exercise} Linear Inverted Pendulum MPC
:label: inverted-pendulum-linear-mpc

- Design an MPC controller for the linearized inverted pendulum.
- For each case, try different cost functions:
  - $\sum \theta^2$
  - $\sum |\theta|$
  - $E_{\text{kinetic}} - E_{\text{potential}}$
:::

::::{solution} inverted-pendulum-linear-mpc
::::

#### Controller

In [ ]:
mpc = do_mpc.controller.MPC(inverted_pendulum_lin)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 50,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
xss = np.array([0.0, 0.0])
distance_cost = casadi.bilin(np.diag([100, 1]), inverted_pendulum_lin.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 1e-4
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

In [ ]:
mpc.setup()

#### Simulation

In [ ]:
%%capture
mpc.reset_history()
linear_simulator.reset_history()
x0 = np.zeros((2, 1))
x0[0] = 0.01
linear_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = linear_simulator.make_step(u0)

In [ ]:
animate_inverted_pendulum_simulation(mpc.data)

#### Evaluation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(2)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation[[1, 3]].reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 500
env = create_inverted_pendulum_environment(max_steps=max_steps)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_inverted_pendulum_simulation(mpc.data)

## Exercise

- Design an MPC Controller for the non-linear inverted pendulum system for two different cases:
  1. Cart at origin and upright Pendulm: Set the reference for the cart position to the origin.
  2. Pendulum Swing-up: Set the initial angle to to $-\pi$ i.e. start with the pendulum at the bottom.
  3. Same as 2 but set the reference for the cart position to the origin.
  4. Make the pendulum rotate as fast as possible.
  
> **Note 1** Use the following to create the environment with initial angle set to -np.pi and cutoff angle to np.inf for second > case.
> ```python
> env = create_inverted_pendulum_environment(cutoff_angle=np.inf, initial_angle=-np.pi)
> ```

> **Note 2**: You can access the inverted pendulum's kinetic, respectively potential, energy using
> `inverted_pendulum.aux["E_kinetic"]`, respectively `inverted_pendulum.aux["E_potential"]`

## Solution

### Swing-up

#### Controller

In [ ]:
mpc = do_mpc.controller.MPC(inverted_pendulum)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 120,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
# energy_cost = 0.1 * inverted_pendulum.aux["E_kinetic"] - 1000 * inverted_pendulum.aux["E_potential"]
energy_cost = -10000 * inverted_pendulum.aux["E_potential"]
terminal_cost = energy_cost
stage_cost = energy_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 0.0
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the position
x_max = env.x_threshold
mpc.bounds["lower", "_x", "position"] = -x_max
mpc.bounds["upper", "_x", "position"] = x_max
# lower and upper bounds of the input
u_max = env.force_max
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

In [ ]:
mpc.setup()

#### Simulation

In [ ]:
%%capture
mpc.reset_history()
nonlinear_simulator.reset_history()
x0 = np.array([0.0, -0.99 * np.pi, 0.0, 0.0])
nonlinear_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = nonlinear_simulator.make_step(u0)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

#### Evaluation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.array([0.0, -0.99 * np.pi, 0.0, 0.0])
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 500
env = create_inverted_pendulum_environment(
    max_steps=max_steps,
    theta_threshold=np.inf,
    theta_initial=-np.pi,
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

# Stochastic Optimal Control Problem

So far we have focused on deterministic systems with no noise or disturbances. This is however not the case for real systems.
The methods discussed in this part deal with the problem of controlling dynamical systems
that are subject to system constraints under uncertainty, which can affect numerous parts of the
problem formulation. The system dynamics in discrete-time is given by:

$$
x_{k+1} = f_t(x_k, u_k, k, w_k, \theta_t)
$$

Where:
- $x_k \in \mathbf{R}^n$ is the system state at time $k$.
- $u_k \in \mathbf{R}^n$ is the applied input at time $k$.
- $w_k$ describes a sequence of random variables corresponding to disturbances or process noise in the system, which are often assumed to be independent and identically distributed (i.i.d.).
- $\theta_t \sim \mathcal{Q}^{\theta_t}$ is a random variable describing the parametric uncertainty of the system, which is therefore constant over time.
- The subscript $t$ is used to emphasize that these quantities represent the true system dynamics or true optimal control problem. 

The true problem therefore relates to the development of an optimal controller for a distribution of systems given by $\mathcal{Q}^{\theta_t}$ under random disturbances $w_k$.

The optimality of the controller is defined with respect to a cost or objective function. In the
presence of random model uncertainties, the cost is often defined as the expectation of a sum of
potentially time-varying stage costs of the states and inputs over a possibly infinite horizon $T$:

$$
J_t = E\left(\sum \limits_{k=0}^{T} g_t(x_k, u_k, k)\right),
$$

where the expected value is taken with respect to all random variables.

## Stochastic Predictive Control

The constrained stochastic optimal control problem can be formulated as:

$$
\begin{array}{ll}
J_t^* = \displaystyle\min_{\pi_{k}} & 
E\left[\sum\limits_{k=0}^{T} g_t(x_k, u_k, k)\right]
\\
\text{subject to} & x_{k + 1}= f_t(x_k, u_k, k, w_k, \theta_t)
\\
& u_k = \pi_k(x_0, \dots, x_k)\\
& \bar{W} = [w_0, \dots, w_{N - 1}] \sim \mathcal{Q}^{\bar{W}}, \theta_t \sim \mathcal{Q}^{\theta_t}\\
& P[\bar{X}] = [x_0, \dots, x_{N}] \in \bar{X}_j  \ge p_j, \forall j = 1, \dots, n_{cx}\\
& P[\bar{U}] = [u_0, \dots, u_{N - 1}] \in \bar{U}_j  \ge p_j, \forall j = 1, \dots , n_{cu}\\
\end{array},
$$

Optimizing over a sequence of control laws $\{\pi_k\}$, which can make use of all information in the
form of state measurements $x_k$ up to time step $k$. Problems of this form are in
general very hard to solve, and direct efforts typically rely on some form of discretization in space
and approximate dynamic programming or reinforcement learning.

A notable exception, similar to what we have seen previously, is linear systems under additive noise and quadratic stage costs in the unconstrained setting, for which an exact solution, such as the standard linear quadratic regulator (LQR).

MPC approximates the previous problem by repeatedly solving a simplified version of the
problem initialized at the currently measured state $x_k$ over a shorter horizon $N$
in a receding- horizon fashion.

We introduce the prediction model:

$$
x_{i+1|k} = f(x_{i|k}, u_{i|k}, i + k, w_{i|k}, \theta),
$$

where $f$ is the prediction dynamics. It typically aims at approximating the true dynamics but often differs, e.g.,
for computational reasons or because a succinct description of the true dynamics is unavailable.

We use the subscript $i|k$ to emphasize predictive quantities, where, e.g., $x_{i|k}$ is the i-step-ahead prediction of the state, initialized at $x_{0|k} = x_k$. 

The most widespread MPC formulations of are nominal MPC schemes, which do not consider any uncertainties in the prediction model but instead rely exclusively on the compensation of uncertainties via feedback and by re-solving the problem at the next sampling instance.

In nominal MPC, the optimization can be performed over control sequences $U = [u_{0|k}, \dots , u_{N - 1|k}]$ rather than policies, resulting in the constrained optimal control problem

$$
\begin{array}{ll}
J^∗ &= \displaystyle\min_{U} g_f(x_{N|k}, u_{N|k}, k + N) + \sum\limits_{i=0}^{N-1} g(x_{i|k}, u_{i|k}, i + k)\\
\text{subject to} & x_{i+1|k} = f(x_{i|k}, u_{i|k}, i + k)\\
& U = [u_{0|k}, \dots, u_{N|k}] \in U_j, \forall j = 1, \dots, n_{cu} \\
& X = [x_{0|k}, \dots, x_{N|k}] \in X_j \forall j = 1, \dots, n_{cx} \\
& x_{N|k} \in X_f\\
& x_{0|k} = x_k\\
\end{array}.
$$

The control law is then implicitly defined through the optimization problem as:

$$
π^{\text{MPC}}(x_k, k) = u_{0|k}^*,
$$

where $u_{0|k}^*$ is the first element of the computed optimal control sequence $U^∗$.

## Control Design Challenges

- Ensuring recursive feasibility and achieving optimality despite a short prediction horizon.
- Satisfying input and state constraints in the presence of uncertainty.
- Ensuring computational tractability by properly reformulating constraints and costs and parameterizing control policies

There is no systematic and universal solution to the third challenge, and often the chosen approach is application dependent. Fortunately, the first and second challenges can be addressed by using data. 

## Robust MPC

- Robust MPC guarantees constraint satisfaction for all uncertain element realizations.
- The model is split into a nominal part and additive uncertainty in a compact set. 
- The controller is designed to be robust against the uncertainty.
- The MPC cost is typically optimized for the nominal system.

##  Multi-Stage MPC

The basic idea for the multi-stage approach is to consider various scenarios, where a scenario is defined by one possible realization of all uncertain parameters at every control instant within the horizon. The family of all considered discrete scenarios can be represented as a tree structure, called the scenario tree

:::{figure} ./_static/images/40_multi_state_mpc.png
:width: 60%
:align: center
Scenario tree representation of the uncertainty
evolution for multi-stage MPC.
:::

- Each node in the tree denotes the possible state of the system at every prediction step.
- The branches represent the different possible realizations of the uncertainty.
- The initial state of the system forms the root node of the tree.
- The root node branches into several nodes in the first stage depending on the number of vertex matrix pairs of the parametric uncertainty.
- All the nodes in the first stage branch again in the second stage.
- The sequence continues until the end of prediction horizon N to form the complete scenario tree.
- A path from the root node to the leaf node represents a scenario.

## Example - Inverted Pendulum

In a real system, usually the model parameters cannot be determined exactly, what represents an important source of uncertainty. In this example, we consider that the mass of the pendulum and that of the cart are not known precisely 
and vary with respect to their nominal value.

### Model

#### Model, States and Control inputs

In [ ]:
model = do_mpc.model.Model("continuous")

pos = model.set_variable(var_type="_x", var_name="position")
theta = model.set_variable(var_type="_x", var_name="theta")
dpos = model.set_variable(var_type="_x", var_name="velocity")
dtheta = model.set_variable(var_type="_x", var_name="dtheta")
u = model.set_variable(var_type="_u", var_name="force")

#### Parameters

In [ ]:
# Certain parameters
ip_parameters = InvertedPendulumParameters()
k = 1 / 3
l = ip_parameters.l
gamma = ip_parameters.gamma
g = ip_parameters.g
mu_p = ip_parameters.mu_p
mu_c = ip_parameters.mu_c

In [ ]:
# Uncertain parameters
m = model.set_variable("_p", "m")
M = model.set_variable("_p", "M")

#### ODE

In [ ]:
numerator = (
    (M + m) * g * casadi.sin(theta)
    - casadi.cos(theta)
    * (gamma * u + m * l * dtheta**2 * casadi.sin(theta) - mu_c * dpos)
    - (M + m) / (m * l) * mu_p * dtheta
)
denominator = (1 + k) * (M + m) * l - m * l * casadi.cos(theta) ** 2
ddtheta = numerator / denominator

In [ ]:
numerator = (
    m * g * casadi.cos(theta) * casadi.sin(theta)
    - (1 + k) * (gamma * u + m * l * dtheta**2 * casadi.sin(theta) - mu_c * dpos)
    - mu_p * dtheta * casadi.cos(theta) / l
)
denominator = m * casadi.cos(theta) ** 2 - (1 + k) * (M + m)
ddpos = numerator / denominator

In [ ]:
model.set_rhs("position", dpos)
model.set_rhs("theta", dtheta)
model.set_rhs("velocity", ddpos)
model.set_rhs("dtheta", ddtheta)

#### Setup

In [ ]:
model.setup()

### Controller

In [ ]:
mpc = do_mpc.controller.MPC(model)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 50,
    "n_robust": 1,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
env = create_inverted_pendulum_environment()
xss = np.array([0.5, 0, 0, 0])
distance_cost = casadi.bilin(np.diag([1, 100, 0, 0]), model.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 0.1
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the position
x_max = 1
mpc.bounds["lower", "_x", "position"] = -x_max
mpc.bounds["upper", "_x", "position"] = x_max
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Parameter Uncertainty

In [ ]:
m_values = ip_parameters.m * np.array([1.0, 1.30, 0.70])
M_values = ip_parameters.M * np.array([1.0, 1.30, 0.70])
mpc.set_uncertainty_values(m=m_values, M=M_values)

#### Setup

In [ ]:
mpc.setup()

### Simulation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(4)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 100
env = create_inverted_pendulum_environment(
    render_mode=render_mode,
    max_steps=max_steps,
    cutoff_angle=np.inf,
    initial_angle=0.99 * np.pi,
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)